In [27]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
from flask import Flask
from flask import jsonify, request
import pymysql
import json
from fake_useragent import UserAgent
import time
import sys
import traceback

In [28]:
# 從免費代理網站取得代理ip
def get_proxy_ip(isHttps):
    r = rq.get("https://www.us-proxy.org/",verify=False)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "lxml")
    trs = soup.select("table.table.table-striped.table-bordered tr")
    metas=[]
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 6:
            ifScheme = tds[6].text
            ip = tds[0].text
            port = tds[1].text
            anonymity = tds[4].text
            proxy = "%s:%s"%(ip, port)
            meta = {
                'proxyIp': proxy,
                'connect_times':0,
                'successful_connect_times':0,
                'qual_ratio':0,
                'isDelete':False,
              }
            if ifScheme == isHttps:
                if ifScheme == "yes":
                    meta['ishttps'] = True
                    meta['proxyIp'] = "https://" + meta['proxyIp']
                elif ifScheme == "no":
                    meta['ishttps'] = False
                    meta['proxyIp'] = "http://" + meta['proxyIp']
                metas.append(meta)
                continue
    return metas

In [29]:
# 從資料庫取出ip
def get_ip_from_db(isHttps,db_settings,qual_ratio=0,connect_times=4):
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            if isHttps=="yes":
                get_ip=f"select * FROM proxy_ip where isHttps = 1 and isDelete is False and ( qual_ratio > {qual_ratio} or ( connect_times < {connect_times} ))"
            elif isHttps=="no":
                get_ip=f"select * FROM proxy_ip where isHttps = 0 and isDelete is False and ( qual_ratio > {qual_ratio} or ( connect_times < {connect_times} ))"
            cursor.execute(get_ip)
            xx = cursor.fetchall()
            des = cursor.description
            data=[]
            for i in xx:
                d = {}
                x=0
                for j in des:
                    d[j[0]]=i[x]
                    x=x+1
                data.append(d)
    except Exception as ex:
        print(ex)
    return data

In [30]:
#我要翻頁!!!!!!   by 更改url
def thzucrawler(domain,forum_number,pages=1, detail = False):
    time_start = time.time()
    now = datetime.date.today()
    articles = []
    s = rq.session()
    proxies_this={}
    proxies=get_proxy_ip("no")
    err = 0
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
        "Accept-Encoding": "gzip, deflate, br", 
        "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
        "Host": "web.pcc.gov.tw",  #目標網站 
        "Sec-Fetch-Dest": "document", 
        "Sec-Fetch-Mode": "navigate", 
        "Sec-Fetch-Site": "same-origin", 
        "Upgrade-Insecure-Requests": "?1", 
        "User-Agent":UserAgent().random
         #使用者代理
    } 
    
    for i in range(1,10000):
        print(i-err)
        url = domain + 'forum-'+ str(forum_number)+'-'+str(i-err)+'.html'  
        try:
            response = rq.get(url,cookies={'over18': '1'},proxies = proxies_this,timeout=3)
            html_doc = response.text
            soup = BeautifulSoup(html_doc, 'lxml')
            if i == 1:
                pages_actual_text=soup.select('div.pg a')[-2].text
                #總頁數
                pages_actual= int(re.search('\d.*',pages_actual_text).group(0))
            page = soup.select("div#pgt div strong")[0].text
            for s in soup.select("div.mn div#threadlist div.bm_c form#moderate table tbody"):
                if (response.status_code == 200) and (re.search('normalthread.*',s['id'])):
                    #requests.get() 的結果是 request.Response 物件, 我們可以先透過該物件的 statu_code 屬性取得 server 回覆的狀態碼
                    #(例如 200 表示正常, 404 表示找不到網頁等), 若狀態碼為 200, 代表正常回應
                    board=s.select("th em a")[0].text
                    name=s.select("th a.s.xst")[0].text
                    url_articles= domain + s.select("th a.s.xst")[0]['href']
                    reply = s.select("a.xi2")[0].text
                    look = s.select("td.num em")[0].text
                    nos_of_d=-1
                    date=s.select("td span")[0].text
                    if re.search('前',date) or re.search('天',date):
                        if re.search('小时',date) or re.search('秒',date) or re.search('分钟',date):
                            date=now
                        elif re.search('(.*)天',date).group(1)=='昨':
                            date=now+datetime.timedelta(-1)
                        elif re.search('(.*)天',date).group(1)=='前':
                            date=now+datetime.timedelta(-2)
                        else:
                            delta=re.search('(.*)天',date).group(1)
                            date=now+datetime.timedelta(int(delta))
                    if detail == True:
                        responsee = rq.get(url_articles,cookies={'over18': '1'})
                        html_docc = responsee.text
                        soupp = BeautifulSoup(html_docc, 'lxml')
                        try:
                            nos_of_d_text = soupp.select('dd p:nth-child(3)')[0].text
                            nos_of_d = re.search(r'下载次数:(.*)',nos_of_d_text)
                            nos_of_d = int(nos_of_d.group(1).strip())
                        except Exception as exx:
                            print("沒有下載次數")
                            nos_of_d = -1
                            pass
                    articles.append({'page': page,'board': board,'title': name,'date': str(date),'url': url_articles,'nos_of_d':nos_of_d,'reply':reply,'look':look})
            if i-err == pages or i-err == pages_actual:
                break
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            print("換一下 proxy")
            err = err + 1
            proxies_this = proxies[err]
            if err == len(proxies):
                err = 0
                proxies=get_proxy_ip("no")
            continue
    print("done")
    time_end = time.time()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    return articles

In [56]:
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "As123459362",
    "db": "pttcrawler",
    "charset": "utf8",
    "autocommit":True
}
def send_db(db_settings, raw_datas,typee):
# import charts
# 資料庫參數設定

    time_start = time.time() #開始計時

    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id="select MAX(UID) FROM log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = "INSERT INTO thzu_data(log_UID, page, title, board, date, url, nos_of_d, type, look ,reply)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)  on DUPLICATE KEY UPDATE log_UID = values(log_UID), page = values(page), title = values(title), board = values(board), date = values(date), url = values(url), nos_of_d = values(nos_of_d), type = values(type), look = values(look), reply = values(reply)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始
            cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "start", "", ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 執行
            for raw_data in raw_datas:
                try:
                    cursor.execute(command, (int(log_UID),int(raw_data["page"]), raw_data["title"], raw_data["board"], raw_data["date"], raw_data["url"], raw_data["nos_of_d"], typee, int(raw_data["look"]), int(raw_data["reply"])))
                except Exception as ex:
                    error_class = ex.__class__.__name__ #取得錯誤類型
                    detail = ex.args[0] #取得詳細內容
                    cl, exc, tb = sys.exc_info() #取得Call Stack
                    lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                    fileName = lastCallStack[0] #取得發生的檔案名稱
                    lineNum = lastCallStack[1] #取得發生的行號
                    funcName = lastCallStack[2] #取得發生的函數名稱
                    errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                    print(errMsg)
                    cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "wrong", str(raw_data), str(ex) + " + " + errMsg))
                    continue
            # 紀錄結束
            cursor.execute(command_log, (datetime.datetime.now(), "thzucrawler", "finish", "", ""))
    except Exception as ex:
        print(ex)
        error_class = ex.__class__.__name__ #取得錯誤類型
        detail = ex.args[0] #取得詳細內容
        cl, exc, tb = sys.exc_info() #取得Call Stack
        lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
        fileName = lastCallStack[0] #取得發生的檔案名稱
        lineNum = lastCallStack[1] #取得發生的行號
        funcName = lastCallStack[2] #取得發生的函數名稱
        errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
        print(errMsg)

    time_end = time.time()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('寫入資料庫，time cost', time_c, 's')

In [60]:
# http://29th.cc/forum-220-5.html
# domain : http://29th.cc/
# forum_number : 220
# pages : 5
# raw_datas=thzucrawler("http://29th.cc/",39,10000)
# send_db(db_settings, raw_datas, "日韩情色(BT)")
raw_datas=thzucrawler("http://29th.cc/",181,10000)
send_db(db_settings, raw_datas,"亚洲無碼原創")
raw_datas=thzucrawler("http://29th.cc/",220,10000)
send_db(db_settings, raw_datas,"亚洲有碼原創")
raw_datas=thzucrawler("http://29th.cc/",182,10000)
send_db(db_settings, raw_datas,"欧美無碼")

D:\Program Files\anaconda3\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1
2
HTTPConnectionPool(host='29th.cc', port=80): Max retries exceeded with url: /forum-181-2.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001E95562D730>, 'Connection to 29th.cc timed out. (connect timeout=3)'))
File "D:\Program Files\anaconda3\lib\site-packages\requests\adapters.py", line 504, in send: [ConnectTimeout] HTTPConnectionPool(host='29th.cc', port=80): Max retries exceeded with url: /forum-181-2.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001E95562D730>, 'Connection to 29th.cc timed out. (connect timeout=3)'))
換一下 proxy
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
12

D:\Program Files\anaconda3\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


D:\Program Files\anaconda3\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
done
寫入資料庫，time cost 27.

In [65]:
# 建立Connection物件
conn = pymysql.connect(**db_settings)
# 建立Cursor物件
with conn.cursor(pymysql.cursors.DictCursor) as cursor: # 創建一個字典游標
    get_url_100="select UID, url FROM thzu_data where nos_of_d = 0 limit 0,100"
    cursor.execute(get_url_100)
    urls = cursor.fetchall()

In [66]:
urls

[{'UID': 1, 'url': 'http://29th.cc/thread-2452832-1-1.html'},
 {'UID': 2, 'url': 'http://29th.cc/thread-2452829-1-1.html'},
 {'UID': 3, 'url': 'http://29th.cc/thread-2452827-1-1.html'},
 {'UID': 4, 'url': 'http://29th.cc/thread-2452825-1-1.html'},
 {'UID': 5, 'url': 'http://29th.cc/thread-2452822-1-1.html'},
 {'UID': 6, 'url': 'http://29th.cc/thread-2452820-1-1.html'},
 {'UID': 7, 'url': 'http://29th.cc/thread-2452818-1-1.html'},
 {'UID': 8, 'url': 'http://29th.cc/thread-2452810-1-1.html'},
 {'UID': 9, 'url': 'http://29th.cc/thread-2452809-1-1.html'},
 {'UID': 10, 'url': 'http://29th.cc/thread-2452807-1-1.html'},
 {'UID': 11, 'url': 'http://29th.cc/thread-2452806-1-1.html'},
 {'UID': 12, 'url': 'http://29th.cc/thread-2452805-1-1.html'},
 {'UID': 13, 'url': 'http://29th.cc/thread-2452804-1-1.html'},
 {'UID': 14, 'url': 'http://29th.cc/thread-2452803-1-1.html'},
 {'UID': 15, 'url': 'http://29th.cc/thread-2452801-1-1.html'},
 {'UID': 16, 'url': 'http://29th.cc/thread-2452800-1-1.html'},
 